# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

from htools import *
from incendio.nlp import FillMaskTransform, GenerativeTransform, \
    ParaphraseTransform

/Users/hmamin/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/hmamin/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/hmamin/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/hmamin/anaconda3/lib/python3.7/site-packages/thinc/neural/_custom_kernels.py:36: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/hmamin/anaconda3/lib/python3.7/site-packages/thinc/neural/_custom_kernels.cu' mode='r' encoding='utf8'>
  SRC = (PWD / "_custom_kernels.cu").open("r", encoding="utf8").read()
/Users/hmamin/anaconda3/lib/pyth

In [3]:
cd_root()

Current directory: /Users/hmamin/incendio


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
TRANSFORMS = {
    'fillmask': FillMaskTransform,
    'paraphrase': ParaphraseTransform,
    'generative': GenerativeTransform
}

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
fm_tfm = TRANSFORMS['fillmask'](n=3)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
@immutify_defaults
def generate(transform, source, dest=None, n=5, text_col='text', id_cols=(),
             nrows=None, tfm_kwargs={}, call_kwargs={}):
    # Use list args like --id_cols '[col1, col2]'
    # Use dict args like --call_kwargs '{"drop_pct": .2, "min_keep": 4}'

    # Load data.
    if isinstance(source, (str, Path)):
        df = pd.read_csv(Path(source), usecols=[text_col] + list(id_cols), 
                         nrows=nrows)
    elif isinstance(source, pd.DataFrame):
        df = source
    else:
        raise TypeError('`source` must be a str/Path or pd.DataFrame.')
        
    # Prepare for output file if necessary.
    if isinstance(dest, (str, Path)):
        dest = Path(dest)
        os.makedirs(dest.parent, exist_ok=True)
    elif dest is not None:
        raise ValueError('`dest` must be a str/Path containing the output '
                         'file name to create, or None if you just want to '
                         'return a df.')

    # For simplicity, we stick to one transform at a time. Slow to load so at
    # least for now, let user pass in the transform itself.
    transform = TRANSFORMS[transform](n=n, **tfm_kwargs) \
        if isinstance(transform, str) else transform
    
    # Generate new variations of input text.
    call_kwargs['flat'] = True
    res = transform(df[text_col].tolist(), **call_kwargs)
    res = pd.DataFrame(res, columns=[text_col])
    
    # Attach identifier columns to output (e.g. we usually want to store 
    # labels and or sample IDs. Most of our augmentation methods make 
    # relatively minor changes to the input so all variations of 1 input 
    # should remain in the same set, usually training.).
    if id_cols:
        df_id = pd.concat([df[col].repeat(res.shape[0] // df.shape[0])
                           for col in id_cols], axis=1).reset_index(drop=True)
        res = pd.concat([df_id, res], axis=1)
        
    # Optionally save output.
    if dest: res.to_csv(dest, index=False)
    return res

In [110]:
# res = generate(fm_tfm, 
#                '/Users/hmamin/msan631/hw3_sentence_pairs/data/train.csv', 
#                'data/tmp.csv',
#                n=3, text_col='question1',
#                id_cols=['id', 'qid1'], 
#                nrows=5,
#                call_kwargs={'errors': 'warn', 'n_mask': 4,
#                             'strategy': 'random'})

res = generate('paraphrase', 
               '/Users/hmamin/msan631/hw3_sentence_pairs/data/train.csv', 
               'data/tmp.csv',
               n=2, text_col='question1',
               id_cols=['id', 'qid1'], 
               nrows=3)

/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60157), raddr=('52.217.48.254', 443)>
  self._sock = None
/Users/hmamin/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('192.168.1.119', 60160), raddr=('52.217.48.254', 443)>
  self._sock = None


In [111]:
res.question1.values

array(['What are the steps to invest in the share market in India?',
       'There is a guide to investing in the share market in India.',
       'What is the story of the diamond?',
       'What is the story of the Diamond?',
       'How can I increase the speed of my internet connection?',
       'What can I do to increase the speed of my internet connection?'],
      dtype=object)